<div style="text-align: center;"> <h1>GROUP F MACHINE LEARNING FOR DATA SCIENCE FINAL PROJECT</h1><> </div>

This Jupyter Notebook will be the main program from our project. When the project is finished, we will be able to use this Jupyter code to generate the presentation slides.

In [35]:
import Scripts #Import all the scripts in the "Scripts" folder
import pandas as pd
import torch 
import numpy as np
import matplotlib as plt

The first step to make is to import our training data and store it into a pandas structure and start the data preprocessing.

In [36]:
dataPath = "./Data/TRAIN_Vehicles_Data.xlsx" #The route to the data
dataT = pd.read_excel(dataPath, na_values=["N", " "]) #After seeing the excel we set the null values as "N" and " "


Step 1) By inspection, remove all the columns that we stimate that will be not useful.

In [37]:
dataT = dataT.drop(columns=["RPAKREP_VEHICLE_HKEY","CHASSIS_NUMBER","OFFICE_MAIN_BRAND","MANUFACTURER",
                            "VEHICLE_TYPE","VARIANT","RIM_KEY","UPHOLSTERY_CODE","ENGINE_ID","TRANSMISSION_TYPE",
                            "COMMISSION_NUMBER","LEASING_START","LEASING_END","LEASING_MILAGE","FINANCING_TYPE_NAME",
                            "PURCHASE_DATE","PRICE_LIST","SOLD_INVOICE_COSTUMER_ID","SOLD_CUSTOMER_ID"])


Step 2) Eliminate the columns with more 40% of its values being null. 

In [38]:

null_percentage = dataT.isnull().mean() * 100 #Multiply the mean by 100 to pass to percentage
threshold = 40 #40% of null entries
columns_to_drop = null_percentage[null_percentage > threshold].index #Extract the index of the columns with more of 40% of its values being null
dataT = dataT.drop(columns=columns_to_drop) #Drop the columns
print("\nDataFrame after dropping columns with more than 40% null values:")
print(dataT)




DataFrame after dropping columns with more than 40% null values:
       COMPANY OFFICE MANUFACTURER_SHORT VEHICLE_GROUP  MODEL_CODE  MILEAGE  \
0         44.0     TA                FOR           ECS         NaN   8600.0   
1         12.0     44                VOL           XC4  536K7170F1      0.0   
2         27.0     33                FOR           TOC          26   1297.0   
3         33.0     K1                  V           POL      AE13GV   6020.0   
4         10.0     96                FOR           FOC          15      0.0   
...        ...    ...                ...           ...         ...      ...   
99066     70.0      4                JAG           EPA         NaN     20.0   
99067     17.0     60                FOR           FIE          13  67748.0   
99068     33.0     KA                  V           AMA      T1BE76   3549.0   
99069     10.0     10                FOR           TNT          85  14117.0   
99070     17.0     61                ALF           STE         Na

Step 3) Statistic description of the data

In [ ]:
pd.set_option('display.max_columns', None) #To see all the columns
dataT.describe()

Step 4) Visualization of the variable type of the attributes

In [ ]:
pd.set_option('display.max_rows', None) #To see all the rows
dataT.dtypes